In [12]:
import os
import json
import pandas as pd
import re
import random
import dice
import jsonpickle
import glob

import jsonpickle.ext.pandas as jsonpickle_pandas
jsonpickle_pandas.register_handlers()

In [13]:
class RangeDict(dict):
    def __getitem__(self, item):
        if not isinstance(item, range):
            for key in self:
                if item in key:
                    return self[key]
            raise KeyError(item)
        else:
            return super().__getitem__(item)

        
class PlunderTable:
    # A collection of multiple entries via dict keys
    def __init__(self, entries=None, entry_keys=None, examples=None):
        
        if entries is not None:
            self.entries = {key: entry for key,entry in zip(entry_keys, entries)}
            self.examples = {key: example for key,example in zip(entry_keys, examples)}
        else:
            self.entries = {}
            self.examples = {}
        
    def __str__(self):
        return "\n".join([f"{key} {entry}" for key,entry in self.entries.items()])
    
    def add_entry(self, entry, ID, example=""):
        self.entries[ID] = entry
        self.examples[ID] = example
    
    def get_entry(self, ID):
        return self.entries[ID]
    
    def get_example(self, ID):
        return self.examples[ID]
    
    def merge_items(self, new_ID):
        items = [item for k,entry in self.entries.items() for item in entry.items]
        entry = Entry(items).merge_items()
        return PlunderTable([entry], [new_ID], [""])
    
    def roll_condition(self, table_dict, for_each=False):
        keys,entries = zip(*[(key,entry.roll_condition(table_dict, for_each)) 
                             for key,entry in self.entries.items()])
        return PlunderTable(entries, keys, [""])
    
    def sort_items(self, deep=True):
        if deep:
            keys,entries = zip(*[(key,entry.sort_items()) for key,entry in self.entries.items()])
        else:
            keys,entries = zip(*self.entries.items())
        keys,entries = zip(*sorted(zip(keys,entries))) 
        return PlunderTable(entries, keys, [""])
    
    def plunder(self, table_dict, ID):
        return self.entries[ID].plunder(table_dict)
    
    def plunder_verbose(self, table_dict, ID):
        print(self.get_entry(ID), "\n")
        plunder = self.plunder(table_dict, ID)
        print(plunder, "\n")
        return plunder
        
        
class Table:
    # A collection of multiple entries via range dicts
    def __init__(self, entries=None, entry_ranges=None):
        self.entries = RangeDict({range(key[0], key[1]): entry for key,entry in zip(entry_ranges, entries)})
        
    def __str__(self):
        return "\n".join([f"{key} {entry}" for key,entry in self.entries.items()])
    
    def rekey(self):
        #self.entries = RangeDict({range(int(key.lstrip("range(").rstrip(")").split(",")[0]),
        #                                int(key.lstrip("range(").rstrip(")").split(",")[1]))
        #                          : entry for key,entry in self.entries.items()})
        self.entries = RangeDict({eval(key): entry for key,entry in self.entries.items()})
    
    def get_entry(self, roll):
        return self.entries[int(roll)]
    
    def plunder(self, table_dict, roll):
        try:
            return self.entries[int(roll)].plunder(table_dict)
        except:
            key_max = max([key[-1] for key in self.entries.keys()])
            if roll > key_max:
                roll = key_max
            else:
                key_min = min([key[0] for key in self.entries.keys()])
                if roll < key_min:
                    roll = key_min
                else:
                    roll -= 1
            return self.plunder(table_dict, roll)
    
    
class Entry:
    # A collection of multiple items
    def __init__(self, items):
        self.items = items.copy()
        
    def __str__(self):
        return ", ".join([str(item) for item in self.items])
    
    def merge_items(self):
        new_items = {}
        for item in self.items:
            # Don't bother with chances. 
            if (item.chance is None) & (item.table_roll is None):
                ID = item.name+str(item.table_roll)+str(item.condition) 
                if ID in new_items.keys():
                    new_items[ID].num_items += item.num_items
                else:
                    new_items[ID] = item
               
        return Entry(list(new_items.values()))
    
    def roll_condition(self, table_dict, for_each=False):
        return Entry([i for s in [item.roll_condition(table_dict, for_each) for item in self.items] for i in s])
    
    def sort_items(self):
        return Entry(sorted(self.items, key=lambda x: x.name))
    
    def plunder(self, table_dict):
        return Entry([i for s in [item.plunder(table_dict) for item in self.items] for i in s])
        
    
class Item:
    def __init__(self, name, chance=None, num_items=1, table_roll=None, condition=None):
        # name can be item name or table reference
        self.name = name
        self.chance = chance
        self.num_items = num_items
        self.table_roll = table_roll
        self.condition = condition
        
    def __str__(self):
        return (f'{self.num_items} {self.name}'
                f'{" ["+self.condition+"]" if self.condition is not None else ""}'
                f'{" ("+self.table_roll+")" if self.table_roll is not None else ""}')
        
    def roll_condition(self, table_dict, for_each=False):
        # conditions: perfect, worn, light damage, moderate damage, heavy damage, broken, ruined
        condition = self.condition
        
        if (condition is None) and (for_each):
            return [Item(self.name, chance=self.chance, num_items=1, table_roll=self.table_roll,
                        condition=table_dict["equipment_wear"].plunder(table_dict, dice.roll("1d20t")).items[0].condition)
                   for i in range(self.num_items)]
        
        if condition is None:
            condition = table_dict["equipment_wear"].plunder(table_dict, dice.roll("1d20t")).items[0].condition
        return [Item(self.name, chance=self.chance, num_items=self.num_items, table_roll=self.table_roll, condition=condition)]          
        
    def plunder(self, table_dict):
        # roll on chance
        if self.chance is not None:
            if random.randint(1,100) < int(self.chance):
                return [Item("nothing")]
        
        # roll num items if not int
        if self.num_items is not None:
            num_items = dice.roll(str(self.num_items)+"t")
        
        # roll on tables for each item
        # return new item in collapsed form
        table_name = "_".join(self.name.split())
        if table_name in table_dict.keys():
            print(f"{table_name}")
            table_roll = "1d100" if self.table_roll is None else self.table_roll
            new_entries = [table_dict[table_name].plunder(table_dict, dice.roll(str(table_roll)+"t")) 
                           for i in range(num_items)]
            new_items = [item for entry in new_entries for item in entry.items]
            return new_items
        else:
            return [Item(self.name, chance=None, num_items=num_items, table_roll=None, condition=self.condition)]
        

In [14]:
def make_tables():
    #dirname = os.path.dirname(__file__)
    #searchname = os.path.join(dirname, 'json', '*')
    #filenames = glob.iglob(searchname)
    filenames = glob.iglob("json/*")
    table_dict = {}
    for filename in filenames:
        print(f"{filename}")
        f = open(filename, "r")
        table_name = re.split('[/.]', filename)[1]
        table_dict[table_name] = jsonpickle.decode(f.read())
        if table_name not in ["plunder"]:
            table_dict[table_name].rekey()
            
    return table_dict

table_dict = make_tables()

json/weapon.json
json/armor.json
json/rare_item.json
json/energy_weapon.json
json/item_quirk.json
json/uncommon_item.json
json/equipment_wear.json
json/random_loot.json
json/common_item.json
json/plunder.json


In [15]:
def plunder(table_dict, ID):
    return table_dict["plunder"].plunder(table_dict, ID)

def vplunder(table_dict, ID):
    return table_dict["plunder"].plunder_verbose(table_dict, ID)

In [16]:
k = "P1"

booty = PlunderTable()

for i in range(5, 1, -1):
    booty.add_entry(plunder(table_dict, k), f"Roll {i+1}")
    
print(booty)

weapon
armor
random_loot
common_item
weapon
armor
weapon
armor
weapon
armor
Roll 6 1 sword, 1 no armor, 1 ration, 1 bonding tape
Roll 5 1 axe, 1 shield, 1 nothing, 1 nothing
Roll 4 1 spear, 1 no armor, 1 nothing, 1 nothing
Roll 3 1 spear, 1 shield, 1 ration, 1 nothing


In [17]:
print(booty.merge_items("Merge 1"))

Merge 1 1 sword, 2 no armor, 2 ration, 1 bonding tape, 1 axe, 2 shield, 5 nothing, 2 spear


In [18]:
print(booty.roll_condition(table_dict))
print("\n")
print(booty.roll_condition(table_dict, for_each=True))

Roll 6 1 sword [worn], 2 no armor [moderate damage], 2 ration [worn], 1 bonding tape [worn]
Roll 5 1 axe [light damage], 2 shield [worn], 5 nothing [worn], 1 nothing [worn]
Roll 4 2 spear [worn], 1 no armor [light damage], 1 nothing [worn], 1 nothing [worn]
Roll 3 1 spear [light damage], 1 shield [ruined], 1 ration [ruined], 1 nothing [light damage]


Roll 6 1 sword [worn], 1 no armor [light damage], 1 no armor [perfect], 1 ration [light damage], 1 ration [moderate damage], 1 bonding tape [worn]
Roll 5 1 axe [worn], 1 shield [moderate damage], 1 shield [ruined], 1 nothing [perfect], 1 nothing [light damage], 1 nothing [light damage], 1 nothing [worn], 1 nothing [perfect], 1 nothing [worn]
Roll 4 1 spear [heavy damage], 1 spear [moderate damage], 1 no armor [worn], 1 nothing [worn], 1 nothing [light damage]
Roll 3 1 spear [worn], 1 shield [light damage], 1 ration [heavy damage], 1 nothing [light damage]


In [19]:
keys, entries = zip(*booty.entries.items())
print(keys)

('Roll 6', 'Roll 5', 'Roll 4', 'Roll 3')


In [20]:
print(booty)
print("\n")
print(booty.sort_items())
print("\n")
print(booty.sort_items(deep=False))

Roll 6 1 sword, 2 no armor, 2 ration, 1 bonding tape
Roll 5 1 axe, 2 shield, 5 nothing, 1 nothing
Roll 4 2 spear, 1 no armor, 1 nothing, 1 nothing
Roll 3 1 spear, 1 shield, 1 ration, 1 nothing


Roll 3 1 nothing, 1 ration, 1 shield, 1 spear
Roll 4 1 no armor, 1 nothing, 1 nothing, 2 spear
Roll 5 1 axe, 5 nothing, 1 nothing, 2 shield
Roll 6 1 bonding tape, 2 no armor, 2 ration, 1 sword


Roll 3 1 spear, 1 shield, 1 ration, 1 nothing
Roll 4 2 spear, 1 no armor, 1 nothing, 1 nothing
Roll 5 1 axe, 2 shield, 5 nothing, 1 nothing
Roll 6 1 sword, 2 no armor, 2 ration, 1 bonding tape


In [21]:
print(table_dict)
for key,table in table_dict.items():
    print(key.upper())
    print(table)

{'weapon': <__main__.Table object at 0x110a07a50>, 'armor': <__main__.Table object at 0x110c04fd0>, 'rare_item': <__main__.Table object at 0x110b65690>, 'energy_weapon': <__main__.Table object at 0x110b656d0>, 'item_quirk': <__main__.Table object at 0x110b52a50>, 'uncommon_item': <__main__.Table object at 0x110a35f10>, 'equipment_wear': <__main__.Table object at 0x110c0be90>, 'random_loot': <__main__.Table object at 0x110c0b990>, 'common_item': <__main__.Table object at 0x110bb88d0>, 'plunder': <__main__.PlunderTable object at 0x110a6a4d0>}
WEAPON
range(1, 2) 1 knife
range(2, 3) 1 club
range(3, 4) 1 spear
range(4, 5) 1 spear
range(5, 6) 1 sword
range(6, 7) 1 axe
range(7, 8) 1 breechloading rifle
range(8, 9) 1 spear
range(9, 10) 1 sword
range(10, 11) 1 revolver
range(11, 12) 1 revolver
range(12, 13) 1 semi-auto pistol
range(13, 14) 1 semi-auto rifle
range(14, 15) 1 shotgun
range(15, 16) 1 monoblade
range(16, 17) 1 submachine gun
range(17, 18) 1 combat rifle
range(18, 19) 1 monoblade
ran

In [24]:
print(jsonpickle.encode(table_dict["weapon"], make_refs=False))

{"py/object": "__main__.Table", "entries": {"py/object": "__main__.RangeDict", "range(1, 2)": {"py/object": "__main__.Entry", "items": [{"py/object": "__main__.Item", "name": "knife", "chance": null, "num_items": "1", "table_roll": null, "condition": null}]}, "range(2, 3)": {"py/object": "__main__.Entry", "items": [{"py/object": "__main__.Item", "name": "club", "chance": null, "num_items": "1", "table_roll": null, "condition": null}]}, "range(3, 4)": {"py/object": "__main__.Entry", "items": [{"py/object": "__main__.Item", "name": "spear", "chance": null, "num_items": "1", "table_roll": null, "condition": null}]}, "range(4, 5)": {"py/object": "__main__.Entry", "items": [{"py/object": "__main__.Item", "name": "spear", "chance": null, "num_items": "1", "table_roll": null, "condition": null}]}, "range(5, 6)": {"py/object": "__main__.Entry", "items": [{"py/object": "__main__.Item", "name": "sword", "chance": null, "num_items": "1", "table_roll": null, "condition": null}]}, "range(6, 7)": {"

In [37]:
for item in dir(table_dict["weapon"]):
    print(f"{item}")#: {table_dict['weapon'].item}")
vars(table_dict['weapon'])

__class__
__delattr__
__dict__
__dir__
__doc__
__eq__
__format__
__ge__
__getattribute__
__gt__
__hash__
__init__
__init_subclass__
__le__
__lt__
__module__
__ne__
__new__
__reduce__
__reduce_ex__
__repr__
__setattr__
__sizeof__
__str__
__subclasshook__
__weakref__
entries
get_entry
plunder
rekey


{'entries': {range(1, 2): <__main__.Entry at 0x110c04f10>,
  range(2, 3): <__main__.Entry at 0x110c04c90>,
  range(3, 4): <__main__.Entry at 0x1109ebed0>,
  range(4, 5): <__main__.Entry at 0x110c04050>,
  range(5, 6): <__main__.Entry at 0x110c048d0>,
  range(6, 7): <__main__.Entry at 0x110c04c10>,
  range(7, 8): <__main__.Entry at 0x110c04f90>,
  range(8, 9): <__main__.Entry at 0x110c04cd0>,
  range(9, 10): <__main__.Entry at 0x110c04390>,
  range(10, 11): <__main__.Entry at 0x110c04310>,
  range(11, 12): <__main__.Entry at 0x110c2d090>,
  range(12, 13): <__main__.Entry at 0x110c2d0d0>,
  range(13, 14): <__main__.Entry at 0x110c2d110>,
  range(14, 15): <__main__.Entry at 0x110c2d190>,
  range(15, 16): <__main__.Entry at 0x110c2d210>,
  range(16, 17): <__main__.Entry at 0x110c2d290>,
  range(17, 18): <__main__.Entry at 0x110c2d250>,
  range(18, 19): <__main__.Entry at 0x110c2dd50>,
  range(19, 20): <__main__.Entry at 0x110c2d590>,
  range(20, 21): <__main__.Entry at 0x110c2dad0>,
  rang

In [38]:
import pickle

In [39]:
pickle.dumps(table_dict["weapon"])

b'\x80\x03c__main__\nTable\nq\x00)\x81q\x01}q\x02X\x07\x00\x00\x00entriesq\x03c__main__\nRangeDict\nq\x04)\x81q\x05(cbuiltins\nrange\nq\x06K\x01K\x02K\x01\x87q\x07Rq\x08c__main__\nEntry\nq\t)\x81q\n}q\x0bX\x05\x00\x00\x00itemsq\x0c]q\rc__main__\nItem\nq\x0e)\x81q\x0f}q\x10(X\x04\x00\x00\x00nameq\x11X\x05\x00\x00\x00knifeq\x12X\x06\x00\x00\x00chanceq\x13NX\t\x00\x00\x00num_itemsq\x14X\x01\x00\x00\x001q\x15X\n\x00\x00\x00table_rollq\x16NX\t\x00\x00\x00conditionq\x17Nubasbh\x06K\x02K\x03K\x01\x87q\x18Rq\x19h\t)\x81q\x1a}q\x1bh\x0c]q\x1ch\x0e)\x81q\x1d}q\x1e(h\x11X\x04\x00\x00\x00clubq\x1fh\x13Nh\x14h\x15h\x16Nh\x17Nubasbh\x06K\x03K\x04K\x01\x87q Rq!h\t)\x81q"}q#h\x0c]q$h\x0e)\x81q%}q&(h\x11X\x05\x00\x00\x00spearq\'h\x13Nh\x14h\x15h\x16Nh\x17Nubasbh\x06K\x04K\x05K\x01\x87q(Rq)h\t)\x81q*}q+h\x0c]q,h\x0e)\x81q-}q.(h\x11X\x05\x00\x00\x00spearq/h\x13Nh\x14h\x15h\x16Nh\x17Nubasbh\x06K\x05K\x06K\x01\x87q0Rq1h\t)\x81q2}q3h\x0c]q4h\x0e)\x81q5}q6(h\x11X\x05\x00\x00\x00swordq7h\x13Nh\x14h\x15h\x16Nh